In [1]:
import pandas as pd
from openpyxl import load_workbook
import openpyxl
import numpy as np
from datetime import date, timedelta

In [2]:
def cell_to_num(char):
    char = char.upper()
    if char.isalpha():
        num = ord(char) - 64
    else:
        num = 0
    return num
print(cell_to_num('a'))

1


In [3]:
# Windows file system
first_file = 'C:/Users/panka/Downloads/Nozzle Sales Report.xlsx'
second_file = 'C:/Users/panka/Downloads/Nozzle Sales Report (1).xlsx'
"""
# linux file system
first_file = '/mnt/c/Users/panka/Downloads/Nozzle Sales Report.xlsx'
second_file = '/mnt/c/Users/panka/Downloads/Nozzle Sales Report (1).xlsx'
"""
wb_check = load_workbook(first_file)

In [4]:
def data_cleaning(path):
    wb = load_workbook(path)
    ws = wb.active
    
    #Unmerge cell
    ws.unmerge_cells('O1:P1')
    ws.unmerge_cells('Q1:R1')
    ws.unmerge_cells('S1:T1')
    ws.unmerge_cells('U1:V1')
    if path[-7] == '1':
        #renaming issue_sale value with dates
        ws['S1'] = str(date.today() - timedelta(days=4))
        ws['T1'] = str(date.today() - timedelta(days=3))
    else:
        ws['S1'] = str(date.today() - timedelta(days=2))    
        ws['T1'] = str(date.today() - timedelta(days=1))
    
    print(ws['S1'].value)
    print(ws['T1'].value)    
    
    #Delete column U:V
    ws.delete_cols(cell_to_num('U'), cell_to_num('V')-cell_to_num('U')+1)
    #Delete column O:R
    ws.delete_cols(cell_to_num('O'), cell_to_num('R')-cell_to_num('O')+1)
    #Delete column K:M
    ws.delete_cols(cell_to_num('K'), cell_to_num('M')-cell_to_num('K')+1)
    #Delete column B:F
    ws.delete_cols(cell_to_num('B'), cell_to_num('F')-cell_to_num('B')+1)
    #Delete second row
    ws.delete_rows(2)
    wb.save(path)

In [5]:
def create_pivot(path):
    #reading excel file using pandas
    old_df = pd.read_excel(path)
    current_wb = load_workbook(path)
    ws = current_wb.active
    
    #filter out the 'NA' value in the product column
    df = old_df[
        (old_df['Product'] == 'XP') |
        (old_df['Product'] == 'HS') |
        (old_df['Product'] == 'MS')
    ]
    
    df.loc[df['Product'] == 'XP', 'Product'] = 'MS'
    
    pvt_all = pd.pivot_table(df,
                            index=["SA"],
                            columns=['Product'],
                            values=[ws['H1'].value, ws['G1'].value],
                            aggfunc=np.sum)
    return pvt_all

In [6]:
data_cleaning(first_file)
data_cleaning(second_file)

ValueError: Cell range O1:P1 is not merged

In [ ]:
pvt = pd.concat([create_pivot(first_file), create_pivot(second_file)], axis=1)
print(pvt)

In [ ]:
with pd.ExcelWriter(first_file, engine='openpyxl') as writer:
    writer.book = load_workbook(first_file)
    pvt.to_excel(writer, 'pivot sheet', index=True)

In [ ]:
df = pd.read_excel(first_file, sheet_name='pivot sheet')
print(df.columns.values)

In [ ]:
cols = df.columns.values
order = [0, 3, 4, 1, 2, 7, 8, 5, 6]
cols = [cols[i] for i in order]
print(cols)

In [ ]:
df = df[cols]
print(df)

In [ ]:
with pd.ExcelWriter(first_file, engine='openpyxl') as writer:
    writer.book = load_workbook(first_file)
    df.to_excel(writer, 'summary', index=False)

In [ ]:
wb = load_workbook(first_file)
ws = wb['summary']
ws.delete_rows(3)
check = 0
for value in ws.iter_rows(min_row=1, max_row=1):
    for cell in value:
        if check % 2 == 0:
            cell.value = ""
        check += 1
        print(cell.value)
# print(df.columns.values)
print(ws['A3'].value)

In [ ]:
"""
data = 'ABCDEFGHI'
for char in data:
    val = char + '1'
    print(ws[val].value)

print(ws.cell(row=1, column=2).value)

#Iterate through columns of the first row
for row in ws.iter_rows(min_row=1, max_row=1):
    for cell in row:
        print(cell.value)
"""

In [ ]:
from exchangelib import Account, Message, Credentials, HTMLBody
from exchangelib import Configuration, DELEGATE
import os

In [ ]:
outlook_user = os.environ.get('OUTLOOK_USER')
outlook_password = os.environ.get('OUTLOOK_PASS')
outlook_server = os.environ.get('OUTLOOK_SERVER')
outlook_email = os.environ.get('OUTLOOK_EMAIL')

print(os.getenv('OUTLOOK_USER'))
print(os.getenv('OUTLOOK_PASS'))

In [ ]:
credentials = Credentials(username=outlook_user,
                         password=outlook_password
                         )
config = Configuration(server=outlook_server,
                      credentials=credentials)
account = Account(primary_smtp_address=outlook_email,
                 config=config,
                 autodiscover=False,
                 access_type=DELEGATE)

In [ ]:
def create_html(ws):
    html = """
    <html><body><h3>Nozzle Sales Report:</h3>
    <table border=1>
    """
    # entering header data in html
    for whole_row in ws.iter_rows(min_row=1, max_row=2):
        html += "<tr>"
        for row in whole_row:
            html += f'<th>{row.value}</th>'
        html += "</tr>"
    # entering table data in html
    for whole_row in ws.iter_rows(min_row=3):
        html += "<tr>"
        for row in whole_row:
            if isinstance(row.value, float):
                html += f'<td>{round(row.value, 2)}</td>'
                print(round(row.value, 2))
            else:
                html += f'<td>{row.value}</td>'
        html += "</tr>"
    html += """
    </table>
    <p>
    <br>
    With Regards<br>
    Pankaj Barnwal<br>
    </p>
    </body>
    </html>
    """
    return html

In [ ]:
msg = Message(
    account=account,
    subject="Nozzle Sales Report - Sambalpur DO",
    body=HTMLBody(create_html(ws)),
    to_recipients=['barnwalp@indianoil.in']
)

msg.send_and_save()